In [24]:
import os

import mxnet as mx
import mxnet.ndarray as nd
from mxnet.gluon.block import HybridBlock
from mxnet.gluon import nn
from mxnet.gluon.model_zoo import model_store
from mxnet import autograd

import numpy as np
from relu_noneg import Relu
import utils

import cv2

In [30]:
class Conv2D(nn.HybridBlock):
    def __init__(self, channels, kernel_size, strides=(1, 1), padding=(0, 0),
                 dilation=(1, 1), groups=1, layout='NCHW',
                 activation=None, use_bias=True, weight_initializer=None,
                 bias_initializer='zeros', in_channels=0, **kwargs):
        super(Conv2D, self).__init__(**kwargs)
        self.conv = nn.Conv2D(channels, kernel_size, strides=strides, padding=padding,
                             dilation=dilation, groups=groups, layout=layout,
                             activation=activation, use_bias=use_bias, weight_initializer=weight_initializer,
                             bias_initializer=bias_initializer, in_channels=in_channels)
        self.output = None
    def hybrid_forward(self, F, x):
        out = self.conv(x)
        self.output = out.asnumpy()
        return out

In [31]:
x = nd.random_normal(shape=(1,3,224,224))
conv = Conv2D(64, kernel_size=11, strides=4, padding=2)
conv.collect_params().initialize()

out = conv(x)

print(conv.output.shape)
print(out.shape)

(1, 64, 55, 55)
(1, 64, 55, 55)


In [32]:
class AlexNet(HybridBlock):
    def __init__(self, classes=1000, **kwargs):
        super(AlexNet, self).__init__(**kwargs)
        with self.name_scope():
            self.features = nn.HybridSequential(prefix='')
            with self.features.name_scope():
                self.features.add(Conv2D(64, kernel_size=11, strides=4, padding=2))
                self.features.add(nn.Activation('relu'))
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2))
                
                self.features.add(Conv2D(192, kernel_size=5, padding=2))
                self.features.add(nn.Activation('relu'))
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2))
                
                self.features.add(Conv2D(384, kernel_size=3, padding=1))
                self.features.add(nn.Activation('relu'))
                
                self.features.add(Conv2D(256, kernel_size=3, padding=1))
                self.features.add(nn.Activation('relu'))
                
                self.features.add(Conv2D(256, kernel_size=3, padding=1))
                self.features.add(nn.Activation('relu'))
                self.features.add(nn.MaxPool2D(pool_size=3, strides=2))
                
                self.features.add(nn.Flatten())

                self.features.add(nn.Dense(4096))
                self.features.add(nn.Activation('relu'))
                self.features.add(nn.Dropout(0.5))

                self.features.add(nn.Dense(4096))
                self.features.add(nn.Activation('relu'))
                self.features.add(nn.Dropout(0.5))

            self.output = nn.Dense(classes)

    def hybrid_forward(self, F, x):
        x = self.features(x)
        x = self.output(x)
        return x

# Constructor
def alexnet(pretrained=False, ctx=mx.cpu(),
            root=os.path.join('~', '.mxnet', 'models'), **kwargs):
    r"""AlexNet model from the `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.

    Parameters
    ----------
    pretrained : bool, default False
        Whether to load the pretrained weights for model.
    ctx : Context, default CPU
        The context in which to load the pretrained weights.
    root : str, default '~/.mxnet/models'
        Location for keeping the model parameters.
    """
    net = AlexNet(**kwargs)
    if pretrained:
        net.load_params(model_store.get_model_file('alexnet', root=root), ctx=ctx)
    return net

In [33]:
alexnet = alexnet(pretrained=True)

In [35]:
def preprocess(data):
    data = mx.image.imresize(data, 256, 256)
    data, _ = mx.image.center_crop(data, (224, 224))
    data = data.astype(np.float32)
    data = data/255
    data = mx.image.color_normalize(data,
                                    mean=mx.nd.array([0.485, 0.456, 0.406]),
                                    std=mx.nd.array([0.229, 0.224, 0.225]))
    data = mx.nd.transpose(data, (2,0,1))
    return data

In [36]:
with open("img/snake.jpg", 'rb') as fp:
    str_image = fp.read()

image = mx.img.imdecode(str_image)
image = preprocess(image)
image = image.expand_dims(axis=0)

out = alexnet(image)
out.argmax(axis=1)


[ 56.]
<NDArray 1 @cpu(0)>

In [37]:
alexnet.features[0].output

array([[[[ -8.83482933e-01,  -3.25704169e+00,  -1.90798795e+00, ...,
            4.74241304e+00,  -1.23616159e+00,  -2.45418096e+00],
         [ -1.91891575e+00,  -5.69753051e-02,   8.64510477e-01, ...,
           -2.28456807e+00,  -8.50221992e-01,  -7.30594158e-01],
         [ -1.32628322e+00,  -2.39729190e+00,  -3.36551237e+00, ...,
           -3.41404963e+00,  -2.13886189e+00,  -3.46683681e-01],
         ..., 
         [ -1.36385083e+00,  -2.63987660e+00,   2.57552087e-01, ...,
           -7.76892364e-01,  -3.92825425e-01,   3.60571682e-01],
         [ -2.02458477e+00,  -2.55291581e-01,  -1.59875631e+00, ...,
           -3.09720206e+00,  -4.40536082e-01,  -3.38562512e+00],
         [ -2.26029336e-01,  -1.74332047e+00,  -1.37486970e+00, ...,
           -3.48535359e-01,  -1.62863731e-02,  -8.89601886e-01]],

        [[ -3.47705436e+00,  -3.39476085e+00,  -2.15162563e+00, ...,
           -2.37417674e+00,  -1.82937527e+00,  -2.14547610e+00],
         [ -3.34120369e+00,  -1.86848402e+00,